#                                                 Overview
This dataset contains every wholesale purchase of liquor in the State of Iowa by retailers for sale to individuals since January 1, 2012. The State of Iowa controls the wholesale distribution of liquor intended for retail sale, which means this dataset offers a complete view of retail liquor sales in the entire state. The dataset contains every wholesale order of liquor by all grocery stores, liquor stores, convenience stores, etc., with details about the store and location, the exact liquor brand and size, and the number of bottles ordered.

In addition to being an excellent dataset for analyzing liquor sales, this is a large and clean public dataset of retail sales data. It can be used to explore problems like stockout prediction, retail demand forecasting, and other retail supply chain problems.

In [54]:
# First, import the relevant modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import requests
import yfinance as yf
import sqlalchemy
from functools import reduce

import json
import requests
from collections import defaultdict

# 1a. Source and load the data
Here I will load iowa_liquor_sales dataset from the IOWA government site using their API.

In [55]:
url = 'https://data.iowa.gov/resource/m3tr-qhgy.json'
r = requests.get(url)
json_data = r.json()

In [66]:
Iowa_Liquor_Sales_API = 'https://data.iowa.gov/resource/m3tr-qhgy.csv'
iowa_liquor_sales = pd.read_csv(Iowa_Liquor_Sales_API, index_col=0)


# Exploring the data

In [57]:
iowa_liquor_sales.describe()

,store,zipcode,county_number,category,vendor_no,itemno,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
count,1000.00000,762.000000,752.000000,1.000000e+03,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,3847.83500,51203.181102,59.389628,1.036496e+06,264.735000,39493.930000,12.29900,897.900000,10.033650,15.052400,9.907000,131.136120,8.683710,2.289170
std,1116.42909,1008.063791,25.706601,5.867247e+04,138.038891,59363.226262,7.65233,510.053423,7.749894,11.624899,17.295592,470.516239,21.153072,5.588366
min,2190.00000,50009.000000,2.000000,1.011100e+06,35.000000,258.000000,1.00000,50.000000,0.900000,1.350000,1.000000,2.700000,0.050000,0.010000
25%,2625.00000,50311.000000,47.000000,1.012100e+06,115.000000,23826.000000,6.00000,600.000000,5.400000,8.100000,3.000000,36.000000,1.750000,0.460000
50%,3848.50000,51040.000000,70.000000,1.031100e+06,260.000000,35318.000000,12.00000,750.000000,7.895000,11.845000,6.000000,66.960000,6.000000,1.580000
75%,4947.00000,52244.750000,78.000000,1.041100e+06,380.000000,40595.250000,12.00000,1000.000000,13.000000,19.500000,12.000000,135.000000,10.500000,2.770000
max,6062.00000,52807.000000,97.000000,1.901200e+06,626.000000,927618.000000,48.00000,1850.000000,99.990000,149.990000,300.000000,13806.000000,525.000000,138.690000


In [58]:
iowa_liquor_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, S31296100034 to INV-20339300007
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 1000 non-null   object 
 1   store                1000 non-null   int64  
 2   name                 1000 non-null   object 
 3   address              762 non-null    object 
 4   city                 762 non-null    object 
 5   zipcode              762 non-null    float64
 6   store_location       227 non-null    object 
 7   county_number        752 non-null    float64
 8   county               752 non-null    object 
 9   category             1000 non-null   int64  
 10  category_name        997 non-null    object 
 11  vendor_no            1000 non-null   int64  
 12  vendor_name          1000 non-null   object 
 13  itemno               1000 non-null   int64  
 14  im_desc              1000 non-null   object 
 15  pack                 

In [59]:
iowa_liquor_sales.shape

(1000, 23)

# 1b. Picking the right the columns
From the documentation of this dataset, I can see that some appropriate columns to look at to answer the brief are:

date
store
name
category
category_name
vendor_no
vendor_name
bottle_volume_ml
state_bottle_cost
state_bottle_retail 
sale_bottles        
sale_dollars         
sale_liters         
sale_gallons 

# 1c. Subsetting accordingly
Here I will select only those columns that I want to work with from this dataset. I'll overwrite the subsets in the original variables.

In [64]:
# Subset the DataFrame object iowa_liquor_sales by selecting just the variables
iowa_liquor_sales = iowa_liquor_sales[['date','store','name','category','category_name','vendor_no',
                                       'vendor_name','bottle_volume_ml',
                                       'state_bottle_cost','state_bottle_retail','sale_bottles',
                                       'sale_dollars','sale_liters','sale_gallons']]
# Exploring the data
# Check the first three entries
iowa_liquor_sales.head(3)

,date,store,name,category,category_name,vendor_no,vendor_name,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
invoice_line_no,,,,,,,,,,,,,,
S31296100034,2016-03-16T00:00:00.000,3762,Wine and Spirits Gallery,1081200,CREAM LIQUEURS,330,Gemini Spirits,1000,7.23,10.85,1,10.85,1.0,0.26
INV-20599500022,2019-07-15T00:00:00.000,4320,Fareway Stores #470 / Perry,1012200,Scotch Whiskies,421,SAZERAC COMPANY INC,1750,10.50,15.75,6,94.50,10.5,2.77
INV-28403900139,2020-07-01T00:00:00.000,2643,Hy-Vee Wine and Spirits / Waterloo,1012400,Irish Whiskies,370,PERNOD RICARD USA,1750,32.48,48.72,2,97.44,3.5,0.92


In [65]:
iowa_liquor_sales.corr()

,store,category,vendor_no,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
store,1.000000,0.011190,0.038951,-0.140694,-0.067437,-0.067438,0.015050,-0.027319,-0.052395,-0.052411
category,0.011190,1.000000,0.085936,0.011895,0.040929,0.040913,-0.005637,-0.003984,-0.007406,-0.007399
vendor_no,0.038951,0.085936,1.000000,-0.011568,-0.043903,-0.043898,0.030469,-0.041295,-0.014964,-0.014942
bottle_volume_ml,-0.140694,0.011895,-0.011568,1.000000,0.258168,0.258175,-0.024018,0.104196,0.231511,0.231592
state_bottle_cost,-0.067437,0.040929,-0.043903,0.258168,1.000000,1.000000,-0.089156,0.156349,0.049341,0.049319
state_bottle_retail,-0.067438,0.040913,-0.043898,0.258175,1.000000,1.000000,-0.089154,0.156348,0.049344,0.049321
sale_bottles,0.015050,-0.005637,0.030469,-0.024018,-0.089156,-0.089154,1.000000,0.722068,0.817815,0.817812
sale_dollars,-0.027319,-0.003984,-0.041295,0.104196,0.156349,0.156348,0.722068,1.000000,0.913472,0.913457
sale_liters,-0.052395,-0.007406,-0.014964,0.231511,0.049341,0.049344,0.817815,0.913472,1.000000,1.000000
sale_gallons,-0.052411,-0.007399,-0.014942,0.231592,0.049319,0.049321,0.817812,0.913457,1.000000,1.000000


# 1d. Choosing a dependent variable
We now need to pick a dependent variable for our regression analysis: a variable whose values we will predict.

'Quality' seems to be as good a candidate as any. Let's check it out. One of the quickest and most informative ways to understand a variable is to make a histogram of it. This gives us an idea of both the center and spread of its values.